In [7]:
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import multivariate_normal

In [8]:
# TODO suturation action, reward state

In [9]:
class DNetwork(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(DNetwork, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        hiden_layer_len_1 = 50
#         hiden_layer_len_2 = 50
        self.hidden_1 = nn.Linear(self.in_dim, hiden_layer_len_1, bias=True)
#         self.hidden_2 = nn.Linear(hiden_layer_len_1, hiden_layer_len_2, bias=True)
        self.output = nn.Linear(hiden_layer_len_1, self.out_dim, bias=True)
        self.activation_on_hidden_1 =  nn.ReLU() # nn.Tanh() # nn.ReLU()
#         self.activation_on_hidden_2 = nn.ReLU() #
        self.activation_on_output = nn.ReLU() # nn.Tanh()  # nn.Softmax()

    def forward(self, x):
#         print(x)
        x = self.hidden_1(x)
#         print(x)
        x = self.activation_on_hidden_1(x)
#         print(x)
#         x = self.hidden_2(x)
#         x = self.activation_on_hidden_2(x)
        x = self.output(x)
#         print(x)
        x = self.activation_on_output(x)
#         print(x)
        # x = self.Norm(x, self.sigma*torch.eye(2)).sample()
        # torch.maximum(torch.minimum(x, torch.tensor(1)), torch.tensor(-1)) # self.activation_on_output(x) #
        return x    

In [10]:
class Utils:
    def __init__(self):
        pass
    
    def prepare_state_action(self, state: list, action: float) -> np.ndarray:
        s = np.array(state)
#         s = s.reshape((s.shape[0], 1))
        a = np.array([action])
#         a = a.reshape((a.shape[0], 1))
        return np.concatenate((s, a), axis=0)

    def prepare_state(self, state: list) -> np.ndarray:
        s = np.array(state)
        return s

In [11]:
utils = Utils()

In [6]:
state = [-0.42193421721458435, -0.015489596903575209, 0.004222722724080086]
action = 2

In [14]:
class Dynamics:
    def __init__(self, state_dim=3, action_dim=1):
        self.model = DNetwork(in_dim=(state_dim+action_dim), out_dim=state_dim)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.model.parameters(), lr=0.002)
    
    def step(self, state: list, action: float) -> list:
        s_a = utils.prepare_state_action(state, action)
        with torch.no_grad():
            state_next = self.model(torch.from_numpy(s_a).float()) # .detach().numpy()
        # list(map(lambda x: x[0], state_next.detach().numpy()))
        state_next = list(state_next.detach().numpy())
        return state_next
    
    def fit(self, states_actions: np.ndarray, next_states: np.ndarray):
        states_actions = torch.from_numpy(states_actions).float()
        next_states = torch.from_numpy(next_states).float()
        self.optimizer.zero_grad()
        guessies = self.model(states_actions)
        loss = self.criterion(guessies, next_states)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [8]:
dynamics = Dynamics()

In [9]:
state = dynamics.step(state, action)

In [49]:
state

[0.0, 0.123333536, 0.0]

In [15]:
class Policy:
    def __init__(self, state_dim=3, action_dim=1):
        self.model = DNetwork(in_dim=state_dim, out_dim=action_dim)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.model.parameters(), lr=0.002)
        
    def get_action(self, state: list) -> float:
        s = utils.prepare_state(state)
        with torch.no_grad():
            action = self.model(torch.from_numpy(s).float()) # .detach().numpy()
        action = action.detach().numpy()[0]
        return action
    
    def fit(self, states: np.ndarray, actions: np.ndarray):
        states = torch.from_numpy(states).float()
        actions = torch.from_numpy(actions).float()
        self.optimizer.zero_grad()
        guessies = self.model(states)
        loss = self.criterion(guessies, actions)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [51]:
policy = Policy()

In [52]:
action = policy.get_action(state)

In [53]:
action

0.0

In [16]:
class Reward:
    def __init__(self, state_dim=3, action_dim=1):
        self.model = DNetwork(in_dim=(state_dim+action_dim), out_dim=1)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.model.parameters(), lr=0.002)
    
    def calculate(self, state: list, action: float) -> float:
        s_a = utils.prepare_state_action(state, action)
        with torch.no_grad():
            reward = self.model(torch.from_numpy(s_a).float()) # .detach().numpy()
        reward = reward.detach().numpy()[0]
        return reward
    
    def fit(self, states_actions: np.ndarray, rewards: np.ndarray):
        states_actions = torch.from_numpy(states_actions).float()
        rewards = torch.from_numpy(rewards).float()
        self.optimizer.zero_grad()
        guessies = self.model(states_actions)
        loss = self.criterion(guessies, rewards)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [16]:
reward_function = Reward()

In [17]:
reward = reward_function.calculate(state, action)

In [18]:
reward

0.15028724

In [9]:
policy = Policy()
reward_function = Reward()
dynamics = Dynamics()

In [17]:
def V(init_state, horizon):
    gamma = 0.9
    value = 0
    state = init_state
    for t in range(1, horizon):
        action = policy.get_action(state)
        reward = reward_function.calculate(state, action)
        value += (gamma**t) * reward
        state = dynamics.step(state, action)
    return value

In [18]:
def E_of_V(init_state, horizon): # TODO horizon не дан
    N = 100
    V_avg = sum([V(init_state, horizon) for _ in range(N)]) / N
    return V_avg

In [17]:
# prepare data

In [1]:
import json

In [2]:
with open('train.json', 'r') as json_file:
    train = json.load(json_file)

In [3]:
episodes_number = len(train)

In [4]:
dynamics_train = []
policy_train = []
reward_train = []

In [5]:
from tqdm import tqdm

In [12]:
for episode in tqdm(range(500)):
    episode_trajectory = train[episode]
    states = episode_trajectory["states"]
    actions = episode_trajectory["actions"]
    rewards = episode_trajectory["rewards"]
    truncated = episode_trajectory["truncated"]
    N = len(rewards)
    
    
    for t in range(N-1):
        s = states[t]
        a = actions[t]
        r = rewards[t]
        s_ = states[t+1]
        
        if t == 0:
            s_a_s = utils.prepare_state_action(s, a)
            s_s = utils.prepare_state(s)
            a_s = np.array([a])
            r_s = np.array([r])
            s_n_s = utils.prepare_state(s_)
        else:
            s_a_s = np.vstack((s_a_s, utils.prepare_state_action(s, a)))
            s_s = np.vstack((s_s, utils.prepare_state(s)))
            a_s = np.vstack((a_s, np.array([a])))
            r_s = np.vstack((r_s, np.array([r])))
            s_n_s = np.vstack((s_n_s, utils.prepare_state(s_)))
            
        dynamics_train.append({"states_actions": s_a_s, "next_states": s_n_s})
        policy_train.append({"states": s_s, "actions": a_s})
        reward_train.append( {"states_actions": s_a_s, "rewards": r_s})
        
    

100%|█████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 58.07it/s]


In [24]:
# fit models

In [19]:
policy = Policy()
reward_function = Reward()
dynamics = Dynamics()

In [20]:
for b in tqdm(range(len(policy_train)//3)):
    policy.fit(policy_train[b]["states"], policy_train[b]["actions"])

100%|███████████████████████████████████████████████████████████████████████████████████| 57378/57378 [00:41<00:00, 1383.40it/s]


In [22]:
for b in tqdm(range(len(reward_train)//5)):
    reward_function.fit(reward_train[b]["states_actions"], reward_train[b]["rewards"])

100%|███████████████████████████████████████████████████████████████████████████████████| 34427/34427 [00:24<00:00, 1384.84it/s]


In [23]:
for b in tqdm(range(len(dynamics_train)//5)):
    dynamics.fit(dynamics_train[b]["states_actions"], dynamics_train[b]["next_states"])

100%|███████████████████████████████████████████████████████████████████████████████████| 34427/34427 [00:25<00:00, 1325.66it/s]


In [ ]:
# submit solution

In [24]:
import json
from random import random


def main():
    # считываем данные
    with open('train.json', 'r') as json_file:
        train = json.load(json_file)

    # посмотрим на информацию, доступную в нулевом эпизоде
    print(train[0].keys())

    # напечатаем возможные действия
    print(set(train[0]['actions']))

    # считываем данные, для которых нужно сделать предсказание
    with open('submit.json', 'r') as json_file:
        submit = json.load(json_file)

    # заполним поле V случайными числами
    for idx, item in enumerate(submit):
        state = item['state']
        item['V'] = V(state, 100)

    # записываем предсказания в файл
    with open('solution.json', 'w') as json_file:
        json.dump(submit, json_file)


In [25]:
main()

dict_keys(['states', 'rewards', 'actions', 'truncated'])
{0, 1, 2}
